In [23]:
#################################
#           battery EV          #
#################################

__version__ = '0.3'

import pandas as pd
import numpy as np
from datetime import timedelta


In [24]:
class price_profiles:
    def __init__(self,request_status):
        self.request_status = request_status
    
    def getProfiles(self):
        import_price1 = pd.read_csv('data/octopus_imprt_price_southern.csv', index_col=0,parse_dates=True,header = 0)
        export_price1 = pd.read_csv('data/octopus_export_price_southern.csv', index_col=0,parse_dates=True,header = 0)

        start_time1 = "2019-1-31 00:00:00"
        end_time1 = "2020-1-30 22:30:00"
        
        start_time2 = "2018-1-31 00:00:00"
        end_time2 = "2019-1-30 22:30:00"
        
        import_price2 =import_price1.loc[start_time1:end_time1]
        export_price2 =export_price1.loc[start_time2:end_time2]
        
        
        import_price3 = import_price2.between_time('09:00:00','16:30:00')
        export_price3 = export_price2.between_time('09:00:00','16:30:00')
        
        for column in import_price3 [['Import Rate (p/kWh)']]:
            import_price = import_price3[column] 
            import_price_value = import_price.values
            
        for column in export_price3 [['Outgoing Agile Rate (p/kWh)']]:
            export_price = export_price3[column]
            export_price_value = export_price.values 
            
        return import_price_value, export_price_value 

In [25]:
class building_profiles:
    def __init__(self,request_status):
        self.request_status = request_status
        
        
    def getProfiles(self):
        excess_profile1 = pd.read_csv('data/Export_vector.csv', index_col=0, parse_dates=True)  # W/kWp
        excess_profile2 = excess_profile1.resample('0.5H').mean()
        excess_profile3 = excess_profile2.append(pd.DataFrame({excess_profile2.columns[0]: np.nan},
                                              index=[(excess_profile2.index[-1] +
                                                      timedelta(minutes=30))]))
        excess_profile4 = excess_profile3.interpolate()
        excess_profile5 = excess_profile4.between_time('09:00:00','16:30:00')
        excess_vector = excess_profile5.values


        import_profile1 = pd.read_csv('data/Import_vector.csv', index_col=0,parse_dates=True)  # W/kWp
        import_profile2 = import_profile1.resample('0.5H').mean()
        import_profile3 = import_profile2.append(pd.DataFrame({import_profile2.columns[0]: np.nan},
                                              index=[(import_profile2.index[-1] +timedelta(minutes=30))]))
        import_profile4 = import_profile3.interpolate()
        import_profile5 = import_profile4.between_time('09:00:00','16:30:00')
        import_vector = import_profile5.values
     
        return import_vector,excess_vector
        

In [26]:
class control_class:
    def __init__(self,request_status):
        self.request_status = request_status
        
    def signalGen(self,import_price_value,export_price_value, max_buy_price, min_sell_price,import_vector,excess_vector):     
        T = len(import_price_value) 
        signal = np.zeros((T, 1))
        
        if self.request_status == 0:  
            for time,price in enumerate(import_price_value):
                if price < max_buy_price:   # Charge/Buys when price < Maximum_Buy_Price
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,price in enumerate(export_price_value):
                if price > min_sell_price :  # Discharge/Sells when price > Minimum_Sell_Price 
                    signal[time] = -1 # Corrects some cases to sell electricity 
                
        elif self.request_status == 1:       
            for time,excess_amount in enumerate(excess_vector):
                if excess_amount > 0:   # Charge the batteries for free when excess generation 
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,import_amount in enumerate(import_vector):
                if import_amount > 0 :  # Discharge when building in need for more electricity  
                    signal[time] = -1 # Corrects some cases to discharge
                #print(time,signal[time])
                
        return signal

In [27]:
class EV_Component:
    def __init__(self):
        self.dispatch_type = "EV Component"
        self.capacity = 0
        self.install_cost = 0
        self.depreciation_cost = 1
        self.lifetime = 15    # Assume all components have a lifespan of 15 years 

In [39]:
class EV_battery(EV_Component):
    def __init__(self, capacity, power, efficiency, dt, install_cost, signal):
        super().__init__()
        self.asset_type = 'EV Battery'
        self.capacity = capacity
        self.power = power * dt   # Convert kW to kWh
        self.eff = efficiency
        self.soc = np.ones(signal.size) * self.capacity
        self.install_cost = install_cost * 100  # p/kWh
        self.depreciation_cost = install_cost/self.lifetime
        self.signal = signal
        
    def getOutput(self, signal,request_status,import_vector,excess_vector,energy_reserved):
        T = len(signal)
        output = np.zeros((T, 1))   # zero array of T rows, 1 column
        SOC_histrory = np.zeros((T, 1)) # to record historic SOC values 
        
        if request_status == 0:
            for time, signal_status in enumerate(signal):
                if time % 16 == 0:           # Reset the soc every morning at 9 am  
                    soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
                else:
                    soc = self.soc[time - 1]           
                if signal[time] == -1 and self.soc[time-1]>energy_reserved:       # Signal indicating selling/ Discharging 
                                                                # Whilst also reserving enough for traveling home
                    output[time] = min(self.power, self.eff*(soc-energy_reserved))
                    self.soc[time] = soc - (1/self.eff)*output[time]
                elif signal[time] == 1:  # Signal indicating buying/ Charging 
                    output[time] = max(-self.power, - (1/self.eff) * (self.capacity - soc))
                    self.soc[time] = soc - self.eff * output[time]        
                else:  # do nothing
                    self.soc[time] = soc
                SOC_histrory[time] = soc
            
        if request_status == 1:
            for time, signal_status in enumerate(signal):
                if time % 16 == 0:           # Reset the soc every morning at 9 am  
                    soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
                else:
                    soc = self.soc[time - 1]           
                if signal[time] == -1 and self.soc[time-1]>energy_reserved:       # Signal indicating discharging
                                                                # Whilst also reserving enough for traveling home
                    output[time] = min(self.power, self.eff*(soc-energy_reserved),self.eff*import_vector[time])
                    self.soc[time] = soc -(1/self.eff)*output[time]
                elif signal[time] == 1:  # Signal indicating Charging 
                    output[time] = max(-self.power, -(1/self.eff)*(self.capacity - soc),-(1/self.eff)*excess_vector[time])
                    self.soc[time] = soc - self.eff * output[time]        
                else:  # do nothing
                    self.soc[time] = soc
                SOC_histrory[time] = soc
                print(time,output[time],self.soc[time])
        return output, SOC_histrory
    
    def getGridCost(self, output, import_price_value, export_price_value,request_status,import_vector,excess_vector,SOC_histrory):
        cost_arr_size = len(output)
        cost_array = np.zeros((cost_arr_size, 1))   # zero array of T rows, 1 column
        system_output = np.zeros((cost_arr_size, 1)) 
        
        
        print('system output')
        for time, output_power in enumerate(output):
            
            if request_status == 0:
                if output[time] > 0:    # Positive output power, selling/discharging
                    cost_array[time] = - export_price_value[time]*output[time] #sell cost is negative
                elif output[time] < 0:  # Negative output power, buying/charging
                    cost_array[time] = - import_price_value[time]*output[time] #buy cost is positive           
                elif output[time] == 0:  # No power output, do nothing 
                    cost_array[time] = 0         
                #print(cost_array[time])
                
            if request_status == 1:
                if output[time] > 0:    # Positive output power, discharging to the buildings
                    # if building demand is not met, some electricity needs to be bought from grid
                    system_output[time] = -(import_vector[time]-output[time])
                    cost_array[time] = import_price_value[time]*(import_vector[time]-output[time]) #buy cost is positive
                elif output[time] < 0:  # Negative output power, charging from the excess of the builidngs 
                    # if building excess is too much, some electricity needs to be sold to grid
                    system_output[time] = -(-excess_vector[time]-output[time])
                    cost_array[time] = export_price_value[time]*(-excess_vector[time]-output[time]) #sell cost is negative         
                elif output[time] == 0 and SOC_histrory[time] == self.capacity:  # No power output due to battery full 
                    system_output[time] = excess_vector[time] # Export all excess 
                    cost_array[time] = -export_price_value[time]*excess_vector[time]          
                else:  # No power output, do nothing 
                    system_output[time] = 0
                    cost_array[time] = 0  
                    
                #print(time,cost_array[time]/100)
                print(time, system_output[time])
                
                
                
        grid_cost = np.sum(cost_array) # in pence 
        grid_cost_GBP = grid_cost/100  # convert to pounds
        print('Grid cost: £ %3.3f' % grid_cost_GBP)
        return grid_cost_GBP
    
    def getUserCost(self, output, import_price_value, export_price_value, request_status):
        cost_arr_size = len(output)
        user_cost_array = np.zeros((cost_arr_size, 1))   # zero array of T rows, 1 column
        
        for time, output_power in enumerate(output):
            if request_status == 1:
                if output[time] > 0 and export_price_value[time] > 7.678:
                    user_cost_array[time] = (export_price_value[time]+7.678)*output[time]
                elif output[time] < 0:  
                    user_cost_array[time] = 0        
                elif output[time] == 0: 
                    user_cost_array[time] = 0         
                #print(user_cost_array[time]/100)
                
        user_cost = np.sum(user_cost_array) 
        user_cost_GBP = user_cost/100  
        print('User cost: £ %3.3f' % user_cost_GBP)       
        return user_cost_GBP
    
    

In [40]:
class V2G_Charger(EV_Component): 
    def __init__(self, install_cost = 0):
        super().__init__()
        self.asset_type = 'EV Charger'
        self.install_cost = install_cost * 100  # p/kWh

In [41]:
#######################
##   Main Programme  ##
#######################
dt = 60/60 # One-hour intervals 
request_status = 1 # Buildings require electricity status

max_buy_price = 4
min_sell_price = 7
energy_reserved = 5

In [42]:
# Price 
price_profiles1 = price_profiles(request_status)
import_price_value, export_price_value = price_profiles1.getProfiles()


In [43]:
# Building Profiles 
building_profiles1 = building_profiles(request_status)
import_vector1,excess_vector1 = building_profiles1.getProfiles()



In [44]:
# Control Signals
signal = control_class(request_status)
signal1 = signal.signalGen(import_price_value, export_price_value, max_buy_price, min_sell_price,import_vector1,excess_vector1)

In [45]:
# Battery
battery_capacity = 60  # kWh
battery_power = 10  # kW
battery_eff = 0.8
battery_cost = 0 
battery_site1 = EV_battery(battery_capacity,battery_power,battery_eff, dt, battery_cost,signal1)
output1,SOC_histrory1 = battery_site1.getOutput(signal1,request_status,import_vector1,excess_vector1,energy_reserved)

0 [10.] 42.5
1 [10.] 30.0
2 [10.] 17.5
3 [10.] 5.0
4 [0.] 5.0
5 [0.] 5.0
6 [0.] 5.0
7 [0.] 5.0
8 [0.] 5.0
9 [0.] 5.0
10 [0.] 5.0
11 [0.] 5.0
12 [0.] 5.0
13 [0.] 5.0
14 [0.] 5.0
15 [0.] 5.0
16 [0.] 55.0
17 [0.] 55.0
18 [0.] 55.0
19 [0.] 55.0
20 [0.] 55.0
21 [0.] 55.0
22 [0.] 55.0
23 [0.] 55.0
24 [0.] 55.0
25 [0.] 55.0
26 [10.] 42.5
27 [10.] 30.0
28 [10.] 17.5
29 [0.] 17.5
30 [0.] 17.5
31 [0.] 17.5
32 [0.] 55.0
33 [0.] 55.0
34 [-6.25] 60.0
35 [-0.] 60.0
36 [-0.] 60.0
37 [-0.] 60.0
38 [-0.] 60.0
39 [-0.] 60.0
40 [-0.] 60.0
41 [-0.] 60.0
42 [-0.] 60.0
43 [-0.] 60.0
44 [-0.] 60.0
45 [-0.] 60.0
46 [-0.] 60.0
47 [0.] 60.0
48 [-6.25] 60.0
49 [-0.] 60.0
50 [-0.] 60.0
51 [-0.] 60.0
52 [-0.] 60.0
53 [-0.] 60.0
54 [-0.] 60.0
55 [-0.] 60.0
56 [-0.] 60.0
57 [-0.] 60.0
58 [-0.] 60.0
59 [-0.] 60.0
60 [-0.] 60.0
61 [-0.] 60.0
62 [-0.] 60.0
63 [0.] 60.0
64 [-6.25] 60.0
65 [-0.] 60.0
66 [-0.] 60.0
67 [-0.] 60.0
68 [-0.] 60.0
69 [-0.] 60.0
70 [-0.] 60.0
71 [-0.] 60.0
72 [-0.] 60.0
73 [-0.] 60.0
74 [-0.] 6

820 [-0.] 60.0
821 [-0.] 60.0
822 [-0.] 60.0
823 [-0.] 60.0
824 [-0.] 60.0
825 [-0.] 60.0
826 [-0.] 60.0
827 [-0.] 60.0
828 [-0.] 60.0
829 [0.] 60.0
830 [0.] 60.0
831 [0.] 60.0
832 [-6.25] 60.0
833 [-0.] 60.0
834 [-0.] 60.0
835 [-0.] 60.0
836 [-0.] 60.0
837 [-0.] 60.0
838 [-0.] 60.0
839 [-0.] 60.0
840 [-0.] 60.0
841 [-0.] 60.0
842 [-0.] 60.0
843 [-0.] 60.0
844 [-0.] 60.0
845 [0.] 60.0
846 [0.] 60.0
847 [0.] 60.0
848 [0.] 55.0
849 [0.] 55.0
850 [0.] 55.0
851 [0.] 55.0
852 [0.] 55.0
853 [0.] 55.0
854 [0.] 55.0
855 [0.] 55.0
856 [0.] 55.0
857 [0.] 55.0
858 [0.] 55.0
859 [0.] 55.0
860 [0.] 55.0
861 [0.] 55.0
862 [0.] 55.0
863 [0.] 55.0
864 [-6.25] 60.0
865 [-0.] 60.0
866 [-0.] 60.0
867 [-0.] 60.0
868 [-0.] 60.0
869 [-0.] 60.0
870 [-0.] 60.0
871 [-0.] 60.0
872 [-0.] 60.0
873 [-0.] 60.0
874 [-0.] 60.0
875 [-0.] 60.0
876 [-0.] 60.0
877 [0.] 60.0
878 [0.] 60.0
879 [0.] 60.0
880 [0.] 55.0
881 [0.] 55.0
882 [0.] 55.0
883 [0.] 55.0
884 [-6.25] 60.0
885 [-0.] 60.0
886 [-0.] 60.0
887 [-0.] 60.0
888

1486 [0.] 60.0
1487 [0.] 60.0
1488 [-6.25] 60.0
1489 [-0.] 60.0
1490 [-0.] 60.0
1491 [-0.] 60.0
1492 [-0.] 60.0
1493 [-0.] 60.0
1494 [-0.] 60.0
1495 [-0.] 60.0
1496 [-0.] 60.0
1497 [-0.] 60.0
1498 [-0.] 60.0
1499 [-0.] 60.0
1500 [-0.] 60.0
1501 [-0.] 60.0
1502 [-0.] 60.0
1503 [0.] 60.0
1504 [-6.25] 60.0
1505 [-0.] 60.0
1506 [-0.] 60.0
1507 [-0.] 60.0
1508 [-0.] 60.0
1509 [-0.] 60.0
1510 [-0.] 60.0
1511 [-0.] 60.0
1512 [-0.] 60.0
1513 [-0.] 60.0
1514 [-0.] 60.0
1515 [-0.] 60.0
1516 [-0.] 60.0
1517 [-0.] 60.0
1518 [-0.] 60.0
1519 [-0.] 60.0
1520 [-6.25] 60.0
1521 [-0.] 60.0
1522 [-0.] 60.0
1523 [-0.] 60.0
1524 [-0.] 60.0
1525 [-0.] 60.0
1526 [-0.] 60.0
1527 [-0.] 60.0
1528 [-0.] 60.0
1529 [-0.] 60.0
1530 [-0.] 60.0
1531 [-0.] 60.0
1532 [-0.] 60.0
1533 [-0.] 60.0
1534 [-0.] 60.0
1535 [-0.] 60.0
1536 [-6.25] 60.0
1537 [-0.] 60.0
1538 [-0.] 60.0
1539 [-0.] 60.0
1540 [-0.] 60.0
1541 [-0.] 60.0
1542 [-0.] 60.0
1543 [-0.] 60.0
1544 [-0.] 60.0
1545 [-0.] 60.0
1546 [-0.] 60.0
1547 [-0.] 60.0
154

2149 [-0.] 60.0
2150 [-0.] 60.0
2151 [-0.] 60.0
2152 [-0.] 60.0
2153 [-0.] 60.0
2154 [-0.] 60.0
2155 [-0.] 60.0
2156 [-0.] 60.0
2157 [-0.] 60.0
2158 [-0.] 60.0
2159 [-0.] 60.0
2160 [-6.25] 60.0
2161 [-0.] 60.0
2162 [-0.] 60.0
2163 [-0.] 60.0
2164 [-0.] 60.0
2165 [-0.] 60.0
2166 [-0.] 60.0
2167 [-0.] 60.0
2168 [-0.] 60.0
2169 [-0.] 60.0
2170 [-0.] 60.0
2171 [-0.] 60.0
2172 [-0.] 60.0
2173 [-0.] 60.0
2174 [-0.] 60.0
2175 [-0.] 60.0
2176 [-6.25] 60.0
2177 [-0.] 60.0
2178 [-0.] 60.0
2179 [-0.] 60.0
2180 [-0.] 60.0
2181 [-0.] 60.0
2182 [-0.] 60.0
2183 [-0.] 60.0
2184 [-0.] 60.0
2185 [-0.] 60.0
2186 [-0.] 60.0
2187 [-0.] 60.0
2188 [-0.] 60.0
2189 [-0.] 60.0
2190 [-0.] 60.0
2191 [-0.] 60.0
2192 [-6.25] 60.0
2193 [-0.] 60.0
2194 [-0.] 60.0
2195 [-0.] 60.0
2196 [-0.] 60.0
2197 [-0.] 60.0
2198 [-0.] 60.0
2199 [-0.] 60.0
2200 [-0.] 60.0
2201 [-0.] 60.0
2202 [-0.] 60.0
2203 [-0.] 60.0
2204 [-0.] 60.0
2205 [-0.] 60.0
2206 [-0.] 60.0
2207 [-0.] 60.0
2208 [-6.25] 60.0
2209 [-0.] 60.0
2210 [-0.] 60.0


2950 [0.] 55.0
2951 [0.] 55.0
2952 [0.] 55.0
2953 [0.] 55.0
2954 [0.] 55.0
2955 [0.] 55.0
2956 [0.] 55.0
2957 [0.] 55.0
2958 [0.] 55.0
2959 [0.] 55.0
2960 [-6.25] 60.0
2961 [-0.] 60.0
2962 [-0.] 60.0
2963 [-0.] 60.0
2964 [-0.] 60.0
2965 [-0.] 60.0
2966 [-0.] 60.0
2967 [-0.] 60.0
2968 [-0.] 60.0
2969 [-0.] 60.0
2970 [-0.] 60.0
2971 [-0.] 60.0
2972 [-0.] 60.0
2973 [-0.] 60.0
2974 [-0.] 60.0
2975 [-0.] 60.0
2976 [0.] 55.0
2977 [0.] 55.0
2978 [-6.25] 60.0
2979 [-0.] 60.0
2980 [-0.] 60.0
2981 [-0.] 60.0
2982 [-0.] 60.0
2983 [-0.] 60.0
2984 [-0.] 60.0
2985 [-0.] 60.0
2986 [-0.] 60.0
2987 [-0.] 60.0
2988 [-0.] 60.0
2989 [-0.] 60.0
2990 [-0.] 60.0
2991 [-0.] 60.0
2992 [-6.25] 60.0
2993 [-0.] 60.0
2994 [-0.] 60.0
2995 [-0.] 60.0
2996 [-0.] 60.0
2997 [-0.] 60.0
2998 [-0.] 60.0
2999 [-0.] 60.0
3000 [-0.] 60.0
3001 [-0.] 60.0
3002 [-0.] 60.0
3003 [-0.] 60.0
3004 [-0.] 60.0
3005 [-0.] 60.0
3006 [-0.] 60.0
3007 [-0.] 60.0
3008 [-6.25] 60.0
3009 [-0.] 60.0
3010 [-0.] 60.0
3011 [-0.] 60.0
3012 [-0.] 6

3492 [-0.] 60.0
3493 [-0.] 60.0
3494 [-0.] 60.0
3495 [-0.] 60.0
3496 [-0.] 60.0
3497 [-0.] 60.0
3498 [-0.] 60.0
3499 [-0.] 60.0
3500 [-0.] 60.0
3501 [-0.] 60.0
3502 [-0.] 60.0
3503 [-0.] 60.0
3504 [-6.25] 60.0
3505 [-0.] 60.0
3506 [-0.] 60.0
3507 [-0.] 60.0
3508 [-0.] 60.0
3509 [-0.] 60.0
3510 [-0.] 60.0
3511 [-0.] 60.0
3512 [-0.] 60.0
3513 [-0.] 60.0
3514 [-0.] 60.0
3515 [-0.] 60.0
3516 [-0.] 60.0
3517 [-0.] 60.0
3518 [-0.] 60.0
3519 [-0.] 60.0
3520 [-6.25] 60.0
3521 [-0.] 60.0
3522 [-0.] 60.0
3523 [-0.] 60.0
3524 [-0.] 60.0
3525 [-0.] 60.0
3526 [-0.] 60.0
3527 [-0.] 60.0
3528 [-0.] 60.0
3529 [-0.] 60.0
3530 [-0.] 60.0
3531 [-0.] 60.0
3532 [-0.] 60.0
3533 [-0.] 60.0
3534 [-0.] 60.0
3535 [-0.] 60.0
3536 [0.] 55.0
3537 [0.] 55.0
3538 [0.] 55.0
3539 [0.] 55.0
3540 [0.] 55.0
3541 [0.] 55.0
3542 [-6.25] 60.0
3543 [-0.] 60.0
3544 [-0.] 60.0
3545 [-0.] 60.0
3546 [-0.] 60.0
3547 [-0.] 60.0
3548 [-0.] 60.0
3549 [-0.] 60.0
3550 [-0.] 60.0
3551 [0.] 60.0
3552 [10.] 42.5
3553 [0.] 42.5
3554 [0.] 

4151 [-0.] 60.0
4152 [-0.] 60.0
4153 [-0.] 60.0
4154 [-0.] 60.0
4155 [-0.] 60.0
4156 [-0.] 60.0
4157 [-0.] 60.0
4158 [-0.] 60.0
4159 [-0.] 60.0
4160 [-6.25] 60.0
4161 [-0.] 60.0
4162 [-0.] 60.0
4163 [-0.] 60.0
4164 [-0.] 60.0
4165 [-0.] 60.0
4166 [-0.] 60.0
4167 [-0.] 60.0
4168 [-0.] 60.0
4169 [-0.] 60.0
4170 [-0.] 60.0
4171 [-0.] 60.0
4172 [-0.] 60.0
4173 [-0.] 60.0
4174 [-0.] 60.0
4175 [0.] 60.0
4176 [0.] 55.0
4177 [0.] 55.0
4178 [0.] 55.0
4179 [0.] 55.0
4180 [-6.25] 60.0
4181 [-0.] 60.0
4182 [-0.] 60.0
4183 [-0.] 60.0
4184 [-0.] 60.0
4185 [-0.] 60.0
4186 [-0.] 60.0
4187 [-0.] 60.0
4188 [-0.] 60.0
4189 [-0.] 60.0
4190 [-0.] 60.0
4191 [0.] 60.0
4192 [-6.25] 60.0
4193 [-0.] 60.0
4194 [-0.] 60.0
4195 [-0.] 60.0
4196 [-0.] 60.0
4197 [-0.] 60.0
4198 [-0.] 60.0
4199 [-0.] 60.0
4200 [-0.] 60.0
4201 [-0.] 60.0
4202 [-0.] 60.0
4203 [-0.] 60.0
4204 [-0.] 60.0
4205 [0.] 60.0
4206 [0.] 60.0
4207 [0.] 60.0
4208 [-6.25] 60.0
4209 [-0.] 60.0
4210 [-0.] 60.0
4211 [-0.] 60.0
4212 [-0.] 60.0
4213 [-0.

4817 [0.] 55.0
4818 [0.] 55.0
4819 [0.] 55.0
4820 [0.] 55.0
4821 [0.] 55.0
4822 [0.] 55.0
4823 [0.] 55.0
4824 [0.] 55.0
4825 [0.] 55.0
4826 [0.] 55.0
4827 [0.] 55.0
4828 [0.] 55.0
4829 [0.] 55.0
4830 [0.] 55.0
4831 [0.] 55.0
4832 [-6.25] 60.0
4833 [-0.] 60.0
4834 [-0.] 60.0
4835 [-0.] 60.0
4836 [-0.] 60.0
4837 [-0.] 60.0
4838 [-0.] 60.0
4839 [-0.] 60.0
4840 [-0.] 60.0
4841 [-0.] 60.0
4842 [-0.] 60.0
4843 [-0.] 60.0
4844 [-0.] 60.0
4845 [0.] 60.0
4846 [0.] 60.0
4847 [0.] 60.0
4848 [0.] 55.0
4849 [0.] 55.0
4850 [0.] 55.0
4851 [0.] 55.0
4852 [-6.25] 60.0
4853 [-0.] 60.0
4854 [-0.] 60.0
4855 [-0.] 60.0
4856 [-0.] 60.0
4857 [-0.] 60.0
4858 [-0.] 60.0
4859 [-0.] 60.0
4860 [-0.] 60.0
4861 [0.] 60.0
4862 [0.] 60.0
4863 [0.] 60.0
4864 [-6.25] 60.0
4865 [-0.] 60.0
4866 [-0.] 60.0
4867 [-0.] 60.0
4868 [-0.] 60.0
4869 [-0.] 60.0
4870 [-0.] 60.0
4871 [-0.] 60.0
4872 [-0.] 60.0
4873 [-0.] 60.0
4874 [-0.] 60.0
4875 [-0.] 60.0
4876 [-0.] 60.0
4877 [0.] 60.0
4878 [0.] 60.0
4879 [0.] 60.0
4880 [10.] 42.

5484 [-0.] 60.0
5485 [-0.] 60.0
5486 [-0.] 60.0
5487 [-0.] 60.0
5488 [0.] 55.0
5489 [0.] 55.0
5490 [0.] 55.0
5491 [0.] 55.0
5492 [0.] 55.0
5493 [0.] 55.0
5494 [0.] 55.0
5495 [0.] 55.0
5496 [-6.25] 60.0
5497 [-0.] 60.0
5498 [-0.] 60.0
5499 [-0.] 60.0
5500 [-0.] 60.0
5501 [0.] 60.0
5502 [0.] 60.0
5503 [0.] 60.0
5504 [0.] 55.0
5505 [0.] 55.0
5506 [0.] 55.0
5507 [0.] 55.0
5508 [0.] 55.0
5509 [0.] 55.0
5510 [-6.25] 60.0
5511 [-0.] 60.0
5512 [-0.] 60.0
5513 [-0.] 60.0
5514 [-0.] 60.0
5515 [-0.] 60.0
5516 [-0.] 60.0
5517 [0.] 60.0
5518 [0.] 60.0
5519 [0.] 60.0
5520 [10.] 42.5
5521 [10.] 30.0
5522 [10.] 17.5
5523 [0.] 17.5
5524 [0.] 17.5
5525 [0.] 17.5
5526 [0.] 17.5
5527 [0.] 17.5
5528 [10.] 5.0
5529 [0.] 5.0
5530 [0.] 5.0
5531 [0.] 5.0
5532 [0.] 5.0
5533 [0.] 5.0
5534 [0.] 5.0
5535 [0.] 5.0
5536 [0.] 55.0
5537 [0.] 55.0
5538 [-6.25] 60.0
5539 [-0.] 60.0
5540 [-0.] 60.0
5541 [-0.] 60.0
5542 [-0.] 60.0
5543 [-0.] 60.0
5544 [-0.] 60.0
5545 [-0.] 60.0
5546 [-0.] 60.0
5547 [-0.] 60.0
5548 [-0.] 6

In [46]:
grid_cost =  battery_site1.getGridCost(output1, import_price_value, export_price_value, request_status, import_vector1,excess_vector1,SOC_histrory1)

system output
0 [-44.93]
1 [-40.53]
2 [-36.13]
3 [-34.1825]
4 [0.]
5 [0.]
6 [0.]
7 [0.]
8 [0.]
9 [0.]
10 [0.]
11 [0.]
12 [0.]
13 [0.]
14 [0.]
15 [0.]
16 [0.]
17 [0.]
18 [0.]
19 [0.]
20 [0.]
21 [0.]
22 [0.]
23 [0.]
24 [0.]
25 [0.]
26 [-9.8275]
27 [-29.655]
28 [-9.8275]
29 [0.]
30 [0.]
31 [0.]
32 [0.]
33 [0.]
34 [132.2125]
35 [276.925]
36 [505.6202778]
37 [734.3155556]
38 [727.5352778]
39 [720.755]
40 [636.795]
41 [552.835]
42 [430.4725]
43 [308.11]
44 [176.37]
45 [44.63]
46 [22.315]
47 [0.]
48 [391.9925]
49 [585.925]
50 [759.8102778]
51 [933.6955556]
52 [1001.5852778]
53 [1069.475]
54 [1035.99]
55 [1002.505]
56 [925.3525]
57 [848.2]
58 [701.5475]
59 [554.895]
60 [382.995]
61 [211.095]
62 [105.5475]
63 [0.]
64 [233.24873457]
65 [384.99]
66 [356.34]
67 [327.69]
68 [314.04]
69 [300.39]
70 [257.435]
71 [214.48]
72 [208.2825]
73 [202.085]
74 [195.3875]
75 [188.69]
76 [169.4875]
77 [150.285]
78 [111.7875]
79 [73.29]
80 [895.895]
81 [954.325]
82 [985.6175]
83 [1016.91]
84 [1019.65]
85 [1022.39

774 [0.]
775 [0.]
776 [0.]
777 [0.]
778 [0.]
779 [0.]
780 [0.]
781 [0.]
782 [0.]
783 [0.]
784 [0.]
785 [0.]
786 [0.]
787 [0.]
788 [-0.31]
789 [-0.62]
790 [86.4902778]
791 [173.7005556]
792 [269.3402778]
793 [364.98]
794 [273.93]
795 [182.88]
796 [91.44]
797 [0.]
798 [0.]
799 [0.]
800 [0.]
801 [0.]
802 [0.]
803 [0.]
804 [214.945]
805 [442.39]
806 [427.08226905]
807 [411.7745381]
808 [445.05726905]
809 [478.34]
810 [314.0975]
811 [149.855]
812 [74.9275]
813 [0.]
814 [0.]
815 [0.]
816 [13.54]
817 [39.58]
818 [68.795]
819 [98.01]
820 [195.3302778]
821 [292.6505556]
822 [318.2077778]
823 [343.765]
824 [265.6275]
825 [187.49]
826 [139.93]
827 [92.37]
828 [46.185]
829 [0.]
830 [0.]
831 [0.]
832 [33.61]
833 [79.72]
834 [269.305]
835 [458.89]
836 [495.2652778]
837 [531.6405556]
838 [608.4702778]
839 [685.3]
840 [599.3325]
841 [513.365]
842 [370.9625]
843 [228.56]
844 [114.28]
845 [0.]
846 [0.]
847 [0.]
848 [0.]
849 [0.]
850 [0.]
851 [0.]
852 [0.]
853 [0.]
854 [0.]
855 [0.]
856 [0.]
857 [0.]
858

1710 [26.495]
1711 [0.]
1712 [1014.5770679]
1713 [1108.5]
1714 [1147.795]
1715 [1187.09]
1716 [1189.0925]
1717 [1191.095]
1718 [1156.2475]
1719 [1121.4]
1720 [1047.89]
1721 [974.38]
1722 [866.1775]
1723 [757.975]
1724 [608.6225]
1725 [459.27]
1726 [294.82]
1727 [130.37]
1728 [827.465]
1729 [939.245]
1730 [970.855]
1731 [1002.465]
1732 [933.305]
1733 [864.145]
1734 [918.5375]
1735 [972.93]
1736 [914.925]
1737 [856.92]
1738 [754.8675]
1739 [652.815]
1740 [530.1775]
1741 [407.54]
1742 [267.4925]
1743 [127.445]
1744 [0.]
1745 [0.]
1746 [0.]
1747 [0.]
1748 [50.6722531]
1749 [113.8445062]
1750 [147.4197531]
1751 [180.995]
1752 [162.9825]
1753 [144.97]
1754 [122.77]
1755 [100.57]
1756 [64.5175]
1757 [28.465]
1758 [14.2325]
1759 [0.]
1760 [631.81209875]
1761 [893.305]
1762 [875.05]
1763 [856.795]
1764 [706.9525]
1765 [557.11]
1766 [428.3625]
1767 [299.615]
1768 [236.9125]
1769 [174.21]
1770 [143.4625]
1771 [112.715]
1772 [70.5025]
1773 [28.29]
1774 [14.145]
1775 [0.]
1776 [42.35771605]
1777 [9

2710 [906.3225]
2711 [841.275]
2712 [708.875]
2713 [576.475]
2714 [481.475]
2715 [386.475]
2716 [320.825]
2717 [255.175]
2718 [194.4725]
2719 [133.77]
2720 [825.4275]
2721 [869.545]
2722 [854.2075]
2723 [838.87]
2724 [775.04]
2725 [711.21]
2726 [723.1775]
2727 [735.145]
2728 [710.6175]
2729 [686.09]
2730 [608.42]
2731 [530.75]
2732 [438.815]
2733 [346.88]
2734 [249.8875]
2735 [152.895]
2736 [964.1975]
2737 [1031.225]
2738 [1057.655]
2739 [1084.085]
2740 [1083.34]
2741 [1082.595]
2742 [1052.6475]
2743 [1022.7]
2744 [959.54]
2745 [896.38]
2746 [803.1175]
2747 [709.855]
2748 [584.2425]
2749 [458.63]
2750 [329.9325]
2751 [201.235]
2752 [871.5725]
2753 [936.765]
2754 [942.645]
2755 [948.525]
2756 [934.3975]
2757 [920.27]
2758 [922.595]
2759 [924.92]
2760 [849.4375]
2761 [773.955]
2762 [667.6425]
2763 [561.33]
2764 [462.3775]
2765 [363.425]
2766 [253.85]
2767 [144.275]
2768 [670.03]
2769 [747.7]
2770 [854.3875]
2771 [961.075]
2772 [938.3825]
2773 [915.69]
2774 [763.98]
2775 [612.27]
2776 [60

3314 [627.75]
3315 [684.415]
3316 [756.3525]
3317 [828.29]
3318 [876.4525]
3319 [924.615]
3320 [861.8]
3321 [798.985]
3322 [678.685]
3323 [558.385]
3324 [456.755]
3325 [355.125]
3326 [245.265]
3327 [135.405]
3328 [831.8975]
3329 [864.1]
3330 [883.21]
3331 [902.32]
3332 [880.915]
3333 [859.51]
3334 [822.3125]
3335 [785.115]
3336 [765.8675]
3337 [746.62]
3338 [696.5025]
3339 [646.385]
3340 [542.54]
3341 [438.695]
3342 [313.1475]
3343 [187.6]
3344 [898.8151543]
3345 [976.16]
3346 [1010.51]
3347 [1044.86]
3348 [1035.41]
3349 [1025.96]
3350 [1005.7975]
3351 [985.635]
3352 [929.085]
3353 [872.535]
3354 [778.7375]
3355 [684.94]
3356 [558.1975]
3357 [431.455]
3358 [295.7925]
3359 [160.13]
3360 [940.1225]
3361 [1017.575]
3362 [1044.725]
3363 [1071.875]
3364 [1066.22]
3365 [1060.565]
3366 [1027.96]
3367 [995.355]
3368 [935.36]
3369 [875.365]
3370 [778.5025]
3371 [681.64]
3372 [565.29]
3373 [448.94]
3374 [317.9375]
3375 [186.935]
3376 [885.3275]
3377 [950.875]
3378 [947.915]
3379 [944.955]
3380 [

4209 [798.6955556]
4210 [898.5477778]
4211 [998.4]
4212 [989.285]
4213 [980.17]
4214 [945.52]
4215 [910.87]
4216 [844.92]
4217 [778.97]
4218 [666.61]
4219 [554.25]
4220 [409.0675]
4221 [263.885]
4222 [131.9425]
4223 [0.]
4224 [604.15506175]
4225 [883.6601235]
4226 [942.58256175]
4227 [1001.505]
4228 [994.2475]
4229 [986.99]
4230 [954.9975]
4231 [923.005]
4232 [850.8375]
4233 [778.67]
4234 [664.3325]
4235 [549.995]
4236 [404.4925]
4237 [258.99]
4238 [129.495]
4239 [0.]
4240 [0.]
4241 [0.]
4242 [155.40157405]
4243 [323.3031481]
4244 [313.05657405]
4245 [302.81]
4246 [257.255]
4247 [211.7]
4248 [178.455]
4249 [145.21]
4250 [108.92]
4251 [72.63]
4252 [36.315]
4253 [0.]
4254 [0.]
4255 [0.]
4256 [402.91632715]
4257 [525.5976543]
4258 [753.21632715]
4259 [980.835]
4260 [980.7375]
4261 [980.64]
4262 [947.1325]
4263 [913.625]
4264 [847.9375]
4265 [782.25]
4266 [667.79]
4267 [553.33]
4268 [398.1325]
4269 [242.935]
4270 [121.4675]
4271 [0.]
4272 [372.58743825]
4273 [535.4298765]
4274 [712.7824382

4967 [565.7]
4968 [521.11]
4969 [476.52]
4970 [434.3425]
4971 [392.165]
4972 [315.4075]
4973 [238.65]
4974 [179.0575]
4975 [119.465]
4976 [784.275]
4977 [844.055]
4978 [861.0075]
4979 [877.96]
4980 [834.605]
4981 [791.25]
4982 [763.85]
4983 [736.45]
4984 [706.645]
4985 [676.84]
4986 [612.8525]
4987 [548.865]
4988 [470.5625]
4989 [392.26]
4990 [276.86]
4991 [161.46]
4992 [785.81083335]
4993 [1000.24]
4994 [1035.1375]
4995 [1070.035]
4996 [1067.835]
4997 [1065.635]
4998 [1027.3325]
4999 [989.03]
5000 [918.8825]
5001 [848.735]
5002 [739.185]
5003 [629.635]
5004 [487.9825]
5005 [346.33]
5006 [200.73]
5007 [55.13]
5008 [0.]
5009 [0.]
5010 [63.8875]
5011 [140.275]
5012 [215.5252778]
5013 [290.7755556]
5014 [386.1377778]
5015 [481.5]
5016 [425.]
5017 [368.5]
5018 [305.4]
5019 [242.3]
5020 [163.1]
5021 [83.9]
5022 [41.95]
5023 [0.]
5024 [-0.9135]
5025 [0.]
5026 [0.]
5027 [0.]
5028 [0.]
5029 [0.]
5030 [0.]
5031 [0.]
5032 [-13.7975]
5033 [-37.595]
5034 [-55.2375]
5035 [-76.534]
5036 [0.]
5037 [0

In [47]:
user_cost =  battery_site1.getUserCost(output1, import_price_value, export_price_value, request_status)

User cost: £ 51.777
